**Aim:** add shapefiles so that we can identify the barrio of each station provided by bicing Json.

In [9]:
import pandas as pd
import requests
import geopandas as gpd
from geopandas import GeoDataFrame
import shapefile as shp
from shapely.geometry import Point, Polygon

In [ ]:
import requests
response = requests.get('http://api.citybik.es/v2/networks/bicing')
data = response.json()

In [4]:
data

{'network': {'company': ['Barcelona de Serveis Municipals, S.A. (BSM)',
   'CESPA',
   'PBSC'],
  'href': '/v2/networks/bicing',
  'id': 'bicing',
  'location': {'city': 'Barcelona',
   'country': 'ES',
   'latitude': 41.3850639,
   'longitude': 2.1734035},
  'name': 'Bicing',
  'stations': [{'empty_slots': 13,
    'extra': {'ebikes': 0, 'has_ebikes': False, 'online': True, 'uid': 428},
    'free_bikes': 11,
    'id': 'bbe41d8c6c52db275f4175fc314fe201',
    'latitude': 41.398389,
    'longitude': 2.196261,
    'name': 'C/ PUJADES, 103',
    'timestamp': '2020-02-12T17:35:48.090000Z'},
   {'empty_slots': 8,
    'extra': {'ebikes': 1, 'has_ebikes': True, 'online': True, 'uid': 427},
    'free_bikes': 11,
    'id': 'f64c50ef1f40d70f28da4b140f274e25',
    'latitude': 41.375336,
    'longitude': 2.168007,
    'name': 'C/ DE SANT PAU, 119/ RONDA SANT PAU',
    'timestamp': '2020-02-12T17:35:48.091000Z'},
   {'empty_slots': 9,
    'extra': {'ebikes': 0, 'has_ebikes': False, 'online': True, 'u

Conver this into a dataframe:

In [ ]:
identifier = [i['id'] for i in data['network']['stations']]
empty_slots = [i['empty_slots'] for i in data['network']['stations']]
e_bikes = [i['extra']['ebikes'] for i in data['network']['stations']]
online = [i['extra']['online'] for i in data['network']['stations']]
lat = [i['latitude'] for i in data['network']['stations']]
long = [i['longitude'] for i in data['network']['stations']]
name = [i['name'] for i in data['network']['stations']]
timestamp = [i['timestamp'] for i in data['network']['stations']]

df = pd.DataFrame()
df['identifier'] = identifier
df['empty_slots'] = empty_slots
df['e_bikes'] = e_bikes
df['online'] = online
df['lat'] = lat
df['long'] = long
df['name'] = name
df['timestamp'] = timestamp

In [13]:
df.head()

,identifier,empty_slots,e_bikes,online,lat,long,name,timestamp
0,bbe41d8c6c52db275f4175fc314fe201,5,1,True,41.398389,2.196261,"C/ PUJADES, 103",2020-02-12T14:55:41.079000Z
1,f64c50ef1f40d70f28da4b140f274e25,20,0,True,41.375336,2.168007,"C/ DE SANT PAU, 119/ RONDA SANT PAU",2020-02-12T14:55:41.080000Z
2,9cb54dcbae0e745227906c99d1059987,6,0,True,41.398305,2.183023,"C/ DE RIBES, 59 B",2020-02-12T14:55:41.081000Z
3,66d72312a5f18984416501ba55b7fd0a,4,0,True,41.376520,2.174900,"C/ DE CERVELLÓ, 5",2020-02-12T14:55:41.083000Z
4,20fd7e2312dd338eea7f562d10c320dc,24,0,True,41.379632,2.192662,PG. MARÍTIM DE LA BARCELONETA,2020-02-12T14:55:41.091000Z


Barrios shapefile:

In [7]:
barrios = gpd.read_file('/users/laurencewilliams/BTS/Projects_2020/Trees_Barcelona/barris.geojson')

In [15]:
pd.set_option("display.max_columns", 50)
barrios.head(2)

,ID_ANNEX,ANNEXDESCR,ID_TEMA,TEMA_DESCR,ID_CONJUNT,CONJ_DESCR,ID_SUBCONJ,SCONJ_DESC,ID_ELEMENT,ELEM_DESCR,NIVELL,NDESCR_CA,NDESCR_ES,NDESCR_EN,TERME,DISTRICTE,BARRI,AEB,SEC_CENS,GRANBARRI,ZUA,AREA_I,LITERAL,PERIMETRE,AREA,ORD_REPRES,CODI_UA,TIPUS_UA,NOM,WEB1,WEB2,WEB3,DOCUMENTA,RANGESCALA,TIPUS_POL,GRUIX_ID,GRUIXDIMEN,ESTIL_ID,ESTIL_QGIS,VALOR1QGIS,VALOR2QGIS,COL_FARCIT,FCOL_DESCR,FHEX_COLOR,COL_DESCR,HEX_COLOR7,geometry
0,01,Grup - I,0104,Unitats Administratives,010413,Barris,01041301,Barri,0104130101,Límit de barri,ADM_03_PL,Límit de barri (polígon),Límite de barrrio (polígono),Neighborhood boundary (polygon),080193,01,01,-,-,01,-,-,01,5521.647,1100286.137,3,01,BARRI,el Raval,http://lameva.barcelona.cat/ciutatvella/ca/hom...,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/document...,None,1-50000,None,6,70.0,0,Sòlid,0,0,41,NULL,#E66900,NULL,#73A88F,"POLYGON ((2.16471 41.38593, 2.16936 41.38554, ..."
1,01,Grup - I,0104,Unitats Administratives,010413,Barris,01041301,Barri,0104130101,Límit de barri,ADM_03_PL,Límit de barri (polígon),Límite de barrrio (polígono),Neighborhood boundary (polygon),080193,01,02,-,-,02,-,-,02,5198.000,815593.938,3,02,BARRI,el Barri Gòtic,http://lameva.barcelona.cat/ciutatvella/ca/hom...,http://www.bcn.cat/estadistica/catala/dades/in...,http://www.bcn.cat/estadistica/catala/document...,None,1-50000,None,6,70.0,0,Sòlid,0,0,41,NULL,#E66900,NULL,#73A88F,"POLYGON ((2.17701 41.38525, 2.17873 41.38396, ..."


Combine the two:

In [16]:
geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
gdf.head(2)

,identifier,empty_slots,e_bikes,online,lat,long,name,timestamp,geometry
0,bbe41d8c6c52db275f4175fc314fe201,5,1,True,41.398389,2.196261,"C/ PUJADES, 103",2020-02-12T14:55:41.079000Z,POINT (2.19626 41.39839)
1,f64c50ef1f40d70f28da4b140f274e25,20,0,True,41.375336,2.168007,"C/ DE SANT PAU, 119/ RONDA SANT PAU",2020-02-12T14:55:41.080000Z,POINT (2.16801 41.37534)


Append Barrio Name:

In [17]:
gdf['barrio'] = pd.Series([])
for i in range(gdf.shape[0]):
    for j in range(barrios.shape[0]):
        if gdf.geometry[i].within(barrios.geometry[j]):
            gdf['barrio'][i] = barrios.NOM[j]
            break

/Users/laurencewilliams/anaconda3/envs/bts36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/laurencewilliams/anaconda3/envs/bts36/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
gdf.head(3)

,identifier,empty_slots,e_bikes,online,lat,long,name,timestamp,geometry,barrio
0,bbe41d8c6c52db275f4175fc314fe201,5,1,True,41.398389,2.196261,"C/ PUJADES, 103",2020-02-12T14:55:41.079000Z,POINT (2.19626 41.39839),el Poblenou
1,f64c50ef1f40d70f28da4b140f274e25,20,0,True,41.375336,2.168007,"C/ DE SANT PAU, 119/ RONDA SANT PAU",2020-02-12T14:55:41.080000Z,POINT (2.16801 41.37534),el Raval
2,9cb54dcbae0e745227906c99d1059987,6,0,True,41.398305,2.183023,"C/ DE RIBES, 59 B",2020-02-12T14:55:41.081000Z,POINT (2.18302 41.39831),el Fort Pienc


We now have a (geo)dataframe with the stations, geometry points and their respective barrio. Each unique identifier of the station can then be associated with the barrio name. We can then update our MySQL table accordingly so we *also* have barrio information, which would allow us to scale our solution to users in multiple barrios.